In [2]:
import pypdfium2 as pdfium
import re
import pandas as pd
import pickle
import openai
import time
import os
import numpy as np
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [7]:
pdf = pdfium.PdfDocument("../02450_Book.pdf")

text_all = ""
for page in pdf:
    textpage = page.get_textpage()
    text_all += " ".join(textpage.get_text_range().splitlines())

In [8]:
text_sentences = list(filter(None, re.split('\. |\? |\! ', text_all)))
text_sentences = [seg for seg in text_sentences if len(seg)>10]

In [9]:
text_segments = []
seg = ""
threshold = 1000 #chars
for s in text_sentences:
    if len(seg)+len(s)>threshold:
        text_segments.append(seg)
        seg = ""
    else:
        seg+=f' {s}'

text_segments = [seg for seg in text_segments if len(seg)>10]

In [10]:
len(text_segments)

671

In [27]:
# Ask ChatGPT to make questions to sections
combinations = []

req_per_min = 0
max_req_per_min = 3
last_time_stamp = time.time()
# For each pair of questions
for i in range(0, len(text_segments)):
    context = text_segments[i]
    
    # Check rate limit
    req_per_min += 1
    while req_per_min>=max_req_per_min:
        time_stamp = time.time()
        if int(time.time()-last_time_stamp) > 60:
            last_time_stamp = time_stamp
            req_per_min = 0
        else:
            time.sleep(10)
        
    # Ask the oracle for label for the context and two questions
    try:
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
            {"role":"system", "content":"You are a system designed to generate questions that can be answered using ANY part of a provided text, and which are specific to it."},
            {"role": "user", "content": f'Please Generate 5 questions CONTEXT: ```{context}```'}
            ]
        )
    except Exception as e:
        print("OPENAI_ERROR:",str(e))
        continue

    # Parse the response to get labels
    res = completion.choices[0].message.content

    # Split and remove index
    split = res.splitlines()
    questions = [re.sub(r'^\d+\.\s', '', q) for q in split]
    #print(questions)
    
    # If 5 questions are not found, assume something went wrong and skip this iteration
    if len(questions) != 5:
        print(f'[Error] questions not generated for {i}')
        continue
    
    # Save data into a dataframe
    combinations += [{'context': context, 'question': question} for question in questions]
    print(f"Questions from round {i+1} saved!")

Questions from round 1 saved!
Questions from round 2 saved!
Questions from round 3 saved!
Questions from round 4 saved!
Questions from round 5 saved!
Questions from round 6 saved!
Questions from round 7 saved!
Questions from round 8 saved!
Questions from round 9 saved!
Questions from round 10 saved!
Questions from round 11 saved!
Questions from round 12 saved!
Questions from round 13 saved!
Questions from round 14 saved!
Questions from round 15 saved!
Questions from round 16 saved!
Questions from round 17 saved!
Questions from round 18 saved!
Questions from round 19 saved!
Questions from round 20 saved!
Questions from round 21 saved!
Questions from round 22 saved!
Questions from round 23 saved!
Questions from round 24 saved!
Questions from round 25 saved!
Questions from round 26 saved!
OPENAI_ERROR: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the reque

In [74]:
# Ask ChatGPT to make questions to failed sections in 02450
#fails = [26, 56, 136, 153, 163, 199, 205, 320, 337, 358, 395, 424, 432, 463, 568, 599, 626, 650, 669]
#fails = [358, 395]
fails = [358]

req_per_min = 0
max_req_per_min = 2
last_time_stamp = time.time()
# For each pair of questions
for f in fails:
    context = text_segments[f]
    
    # Check rate limit
    req_per_min += 1
    while req_per_min>=max_req_per_min:
        time_stamp = time.time()
        if int(time.time()-last_time_stamp) > 59:
            last_time_stamp = time_stamp
            req_per_min = 0
        else:
            time.sleep(15)
        
    # Ask the oracle for label for the context and two questions
    try:
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
            {"role":"system", "content":"You are a system designed to generate questions that can be answered using ANY part of a provided text, and which are specific to it."},
            {"role": "user", "content": f'Please Generate 5 questions CONTEXT: ```{context}```'}
            ]
        )
    except Exception as e:
        print("OPENAI_ERROR:",str(e))
        continue

    # Parse the response to get labels
    res = completion.choices[0].message.content

    # Split and remove index
    split = res.splitlines()
    questions = [re.sub(r'^\d+\.\s', '', q) for q in split]
    #print(questions)
    
    # If 5 questions are not found, assume something went wrong and skip this iteration
    if len(questions) != 5:
        print(f'[Error] questions not generated for {f}')
        print(questions)
        continue
    
    # Save data into a dataframe
    fail_combinations += [{'context': context, 'question': question} for question in questions]
    print(f"Questions from round {f} saved!")

Questions from round 358 saved!


In [161]:
#df = pd.DataFrame(columns=['context', 'question'], data = combinations)

with open('./data/final_2450.pkl', 'wb') as fp:
    pickle.dump(df, fp)

In [75]:
df_fail = pd.DataFrame(columns=['context', 'question'], data = fail_combinations)

with open('./data/fails3_2450.pkl', 'wb') as fp:
    pickle.dump(df_fail, fp)

In [4]:
with open('./data/fails3_2450.pkl', 'rb') as f:
    df_fail = pickle.load(f)

In [157]:
# 26, 56, 136, 153, 163, 199, 205, 320, 337, 424, 432, 463, 568, 599, 626, 650, 669, 395, 358
text_segments[359]

' Root mean square error (rmse) for the train\ufffeing and test set when using an artificial neural network with three hidden units to predict the Area of an island based only on the four attributes x1, x2, x5 and x6 using the hold-out method with 40 % of the observations hold-out for testing Attribute description Abbrev x1 Number of plant species Plants x2 Number of endemic plant species E-Plants x3 Area of island (in km2 ) Area x4 Max elevation above sea-level (in m) Elev x5 Distance to nearest island (in km) DistNI x6 Distance to Santa Cruz Island (in km) StCruz x7 Area of adjacent island (in km2 ) AreaNI Table 10.7 The seven attributes of the data on a selection of 29 of the Gal´apagos islands A Neither forward nor backward selection will identify the optimal feature combination for this problem B Forward selection will result in a better model being selected than using backward selection C Backward selection will terminate at the model that includes the features x1, x2, and x6'

In [152]:
df_fail.loc[90,'context']

' D For least squares linear regression the test error will always decrease as we include more attributes in the model E Don’t know Question 4: We would like to fit an artificial neural network (ANN) model to the Galagapos dataset shown in Table 10.7 To reduce the computational cost of fitting the models it was decided to not include Elev, i.e x4, and AreaNI, i.e x7 in the ANN models We there\ufffefore only consider x1, x2, x5 and x6 in order to predict Area, i.e x3 An artificial neural network with three hid\ufffeden units is applied to the data with these four attributes and trained using different combinations of the four at\ufffetributes x1, x2, x5 and x6 Table 10.6 gives the training and test performance in terms of root mean squared error (rmse) of the ANN for different combinations of the con\ufffesidered attributes Which one of the following statements is correct'

In [156]:
df.loc[1745,'context']

' Root mean square error (rmse) for the train\ufffeing and test set when using an artificial neural network with three hidden units to predict the Area of an island based only on the four attributes x1, x2, x5 and x6 using the hold-out method with 40 % of the observations hold-out for testing Attribute description Abbrev x1 Number of plant species Plants x2 Number of endemic plant species E-Plants x3 Area of island (in km2 ) Area x4 Max elevation above sea-level (in m) Elev x5 Distance to nearest island (in km) DistNI x6 Distance to Santa Cruz Island (in km) StCruz x7 Area of adjacent island (in km2 ) AreaNI Table 10.7 The seven attributes of the data on a selection of 29 of the Gal´apagos islands A Neither forward nor backward selection will identify the optimal feature combination for this problem B Forward selection will result in a better model being selected than using backward selection C Backward selection will terminate at the model that includes the features x1, x2, and x6'

In [158]:
for i in range(5):
    n = round(1744.1+i*0.2,1)
    c = i+90
    df.loc[n] = df_fail.loc[c, 'context'], df_fail.loc[c, 'question']
df

,context,question
0.0,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the purpose of the Introduction to Mac...
1.0,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the size of the data matrix X?
2.0,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the difference between the dependent v...
3.0,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the range of values that the class ind...
4.0,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the purpose of the attributeNames cell...
...,...,...
1744.1,D For least squares linear regression the tes...,What is the purpose of fitting an artificial n...
1744.3,D For least squares linear regression the tes...,Which attributes were excluded from the ANN mo...
1744.5,D For least squares linear regression the tes...,Which attributes were considered in the ANN mo...
1744.7,D For least squares linear regression the tes...,How many hidden units were used in the ANN model?


In [159]:
df = df.sort_index().reset_index(drop=True)

In [160]:
df

,context,question
0,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the purpose of the Introduction to Mac...
1,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the size of the data matrix X?
2,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the difference between the dependent v...
3,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the range of values that the class ind...
4,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the purpose of the attributeNames cell...
...,...,...
3350,"Rex Bryce, James A Hagans, Thomas C Chalmers,...",What is the title of the article published in ...
3351,"Rex Bryce, James A Hagans, Thomas C Chalmers,...","In which year was the article ""On the theory o..."
3352,"Rex Bryce, James A Hagans, Thomas C Chalmers,...","Who are the authors of the article ""Cluster en..."
3353,"Rex Bryce, James A Hagans, Thomas C Chalmers,...",What is the name of the book published by Addi...
